# Morphing

> This module apply morphing to a template source from landmarks. Morphing from pygem library.


In [1]:
#| default_exp morph

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
import os
import numpy as np
import click
from argparse import ArgumentParser

In [4]:
#| export
from infepy.rbf import RBF

In [5]:
#| export
from infepy.preprocessing import read_landmarks, read_nodes, _check_landmarks, write_output
from infepy.utils import read_toml, multiple_targets, _merge_path, to_ls_dyna, read_csv_file,from_df_to_np

In [10]:
#| export
def morphing(source_landmarks: np.ndarray,  # Landmarks of the source mesh
             target_landmarks: np.ndarray,  # Target Landmarks
             source_mesh: np.ndarray, # Coordinates of the mesh/ mesh 
             func,
             smoothing):
    "Morph the target mesh with RBF function - Thin Plate Spine."
    basis_function = func
    
    rbf = RBF(original_control_points=from_df_to_np(source_landmarks) , 
              deformed_control_points= from_df_to_np(target_landmarks),
              func= basis_function,
              smoothing= smoothing)

    return rbf(from_df_to_np(source_mesh))

## CALL from Command Line:
- move into infepy/infepy directory
- type python -m morph

In [11]:
#| export
parser = ArgumentParser(description="Morphing function")
parser.add_argument("--function", 
                    type=str,
                    default='thin_plate_spline',
                    help="Basis Function for morphing")
parser.add_argument("--smoothing", 
                    type=float,
                    default=0.0,
                    help="Smoothing Factor")
        
def do_morphing(func, smoothing):
    config = read_toml()
    source_landmarks = read_landmarks(_merge_path(config['source']['path'],config['source']['filename_landmarks']))
    template_mesh = read_nodes(_merge_path(config['source']['path'],config['source']['filename_mesh']))
    
    if multiple_targets() == False: # single target
        target_landmarks = read_landmarks(_merge_path(config['target']['path'],config['target']['filename_landmarks']))
        _check_landmarks(source_landmarks, target_landmarks)
        morphed_mesh = morphing(source_landmarks, target_landmarks, template_mesh,func, smoothing)
        write_output(morphed_mesh,
                     _merge_path(config['target']['path'], 'morphed.key'),
                     _merge_path(config['source']['path'], config['source']['filename_mesh']))
    else:
        targets_folder = multiple_targets()
        for folder in targets_folder:
            folder_path = os.path.join(config['target']['path'],folder)
            target_landmarks = read_landmarks(_merge_path(folder_path,config['target']['filename_landmarks']))
            _check_landmarks(source_landmarks, target_landmarks)
            morphed_mesh = morphing(source_landmarks, target_landmarks, template_mesh,func, smoothing)
            write_output(morphed_mesh,
                      _merge_path(folder_path,'morphed.key'),
                      _merge_path(config['source']['path'],config['source']['filename_mesh']))
    return

In [13]:
#| export
if __name__ == "__main__":
    args, unknown = parser.parse_args()
    do_morphing(args.function, args.smoothing)

AttributeError: 'Namespace' object has no attribute 'function'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()